In [1]:
import pandas as pd

df = pd.read_csv("MovieReview.csv")
display(df.head())
print(df.shape)

df = df.drop('sentiment', axis=1)

,sentiment,review
0,Positive,With all this stuff going down at the moment w...
1,Positive,'The Classic War of the Worlds' by Timothy Hin...
2,Negative,The film starts with a manager (Nicholas Bell)...
3,Negative,It must be assumed that those who praised this...
4,Positive,Superbly trashy and wondrously unpretentious 8...


(25000, 2)


In [3]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp311-cp311-macosx_10_9_x86_64.whl.metadata (40 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp311-cp311-macosx_10_9_x86_64.whl (287 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# nltk.download()
stop_words = stopwords.words('english')

# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!]+", " ", w)
    w = re.sub(r'\b\w{0,2}\b', '', w)

    # remove stopword
    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

df.review = df.review.apply(lambda x :preprocess_sentence(x))
df.head()

,review
0,stuff going moment started listening music wat...
1,classic war worlds timothy hines entertaining ...
2,film starts manager nicholas bell giving welco...
3,must assumed praised film greatest filmed oper...
4,superbly trashy wondrously unpretentious explo...


In [6]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df.review)

In [14]:
word2idx = tokenizer.word_index
idx2word = tokenizer.index_word
vocab_size = tokenizer.num_words

In [15]:
vocab_size

10000

In [ ]:
import pickle
with open('vocab_infor.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([word2idx, idx2word, vocab_size], f)

In [ ]:
import hickle as hkl
# write variables to filename [a,b,c can be of any size]
filename = 'vocab_info'
hkl.dump([word2idx, idx2word, vocab_size], filename)

# load variables from filename
word2idx, idx2word, vocab_size = hkl.load(filename)

In [7]:
import numpy as np


def sentenceToData(tokens, WINDOW_SIZE):
    window = np.concatenate((np.arange(-WINDOW_SIZE,0),np.arange(1,WINDOW_SIZE+1)))
    X,Y=([],[])
    for word_index, word in enumerate(tokens) :
        if ((word_index - WINDOW_SIZE >= 0) and (word_index + WINDOW_SIZE <= len(tokens) - 1)) :
            X.append(word)
            Y.append([tokens[word_index-i] for i in window])
    return X, Y


WINDOW_SIZE = 5

X, Y = ([], [])
for review in df.review:
    for sentence in review.split("."):
        word_list = tokenizer.texts_to_sequences([sentence])[0]
        if len(word_list) >= WINDOW_SIZE:
            Y1, X1 = sentenceToData(word_list, WINDOW_SIZE//2)
            X.extend(X1)
            Y.extend(Y1)
    
X = np.array(X).astype(int)
y = np.array(Y).astype(int).reshape([-1,1])

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D

embedding_dim = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GlobalAveragePooling1D())
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, batch_size = 128, epochs=50)

Epoch 1/50
12163/12163 [==============================] - 515s 42ms/step - loss: 7.6543 - accuracy: 0.0313
Epoch 2/50
12163/12163 [==============================] - 469s 39ms/step - loss: 6.9537 - accuracy: 0.0581
Epoch 3/50
12163/12163 [==============================] - 412s 34ms/step - loss: 6.5205 - accuracy: 0.0754
Epoch 4/50
12163/12163 [==============================] - 438s 36ms/step - loss: 6.1902 - accuracy: 0.0886
Epoch 5/50
12163/12163 [==============================] - 521s 43ms/step - loss: 5.9204 - accuracy: 0.1005
Epoch 6/50
12163/12163 [==============================] - 527s 43ms/step - loss: 5.6976 - accuracy: 0.1111
Epoch 7/50
12163/12163 [==============================] - 447s 37ms/step - loss: 5.5147 - accuracy: 0.1208
Epoch 8/50
12163/12163 [==============================] - 413s 34ms/step - loss: 5.3656 - accuracy: 0.1303
Epoch 9/50
12163/12163 [==============================] - 412s 34ms/step - loss: 5.2435 - accuracy: 0.1388
Epoch 10/50
12163/12163 [============

In [ ]:
model.save("word2vec.h5") 

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
import pickle

In [ ]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
import pickle

embedding_dim = 300
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GlobalAveragePooling1D())
model.add(Dense(vocab_size, activation='softmax'))

model.load_weights("word2vec.h5")

In [12]:
model.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 300) dtype=float32, numpy=
 array([[-1.6336739e-02, -3.2969713e-03,  2.2689249e-02, ...,
          3.8612876e-02,  8.3623528e-03, -4.6237852e-02],
        [ 4.7363630e-01,  8.3849803e-03,  2.8525680e-02, ...,
         -1.0739079e-01,  1.0848190e-01, -2.3038709e-01],
        [ 1.4934343e-01, -1.8013370e-01, -4.6365210e-01, ...,
         -2.0404343e-01,  1.5457350e-01, -2.9626891e-01],
        ...,
        [-1.6274743e+00,  5.7811457e-01,  3.5598168e-01, ...,
          2.0314949e+00,  7.2257441e-01, -4.6763816e-01],
        [-2.0888686e-01,  1.3393493e+00,  1.2781122e+00, ...,
         -3.4095354e+00, -9.4755644e-01,  1.2645215e+00],
        [-1.3592587e+00, -2.6635106e+00,  3.4834404e+00, ...,
         -4.9866799e-01, -5.7168049e-01,  1.6844219e+00]], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(300, 10000) dtype=float32, numpy=
 array([[-0.76584256, -0.10200101, -0.19614986, ...,  0.5564938 ,
         -1.3159206 ,  0.53

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 300)         3000000   
                                                                 
 global_average_pooling1d_1   (None, 300)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 10000)             3010000   
                                                                 
Total params: 6,010,000
Trainable params: 6,010,000
Non-trainable params: 0
_________________________________________________________________


In [13]:
with open('vocab_infor.pkl','rb') as f:  # Python 3: open(..., 'rb')
    word2idx, idx2word, vocab_size = pickle.load(f)
vocab_size

10000

In [9]:
pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/hui/anaconda3/envs/tensorflow/lib/python3.11/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from tensorflow.keras.models import load_model
import pickle

model = load_model('word2vec.h5')

with open('vocab_infor.pkl','rb') as f:  # Python 3: open(..., 'rb')
    word2idx, idx2word, vocab_size = pickle.load(f)

In [4]:
# Similarity is a metric which measures the distance between two words. This distance represents the way 
# words are related to each other
vectors = model.layers[0].trainable_weights[0].numpy()
import numpy as np
from sklearn.preprocessing import Normalizer

def dot_product(vec1, vec2):
    return np.sum((vec1*vec2))

def cosine_similarity(vec1, vec2):
    return dot_product(vec1, vec2)/np.sqrt(dot_product(vec1, vec1)*dot_product(vec2, vec2))

def find_closest(word_index, vectors, number_closest):
    list1=[]
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if not np.array_equal(vector, query_vector):
            dist = cosine_similarity(vector, query_vector)
            list1.append([dist,index])
    return np.asarray(sorted(list1,reverse=True)[:number_closest])

def compare(index_word1, index_word2, index_word3, vectors, number_closest):
    list1=[]
    query_vector = vectors[index_word1] - vectors[index_word2] + vectors[index_word3]
    normalizer = Normalizer()
    query_vector =  normalizer.fit_transform([query_vector], 'l2')
    query_vector= query_vector[0]
    for index, vector in enumerate(vectors):
        if not np.array_equal(vector, query_vector):
            dist = cosine_similarity(vector, query_vector)
            list1.append([dist,index])
    return np.asarray(sorted(list1,reverse=True)[:number_closest])

def print_closest(word, number=10):
    index_closest_words = find_closest(word2idx[word], vectors, number)
    for index_word in index_closest_words :
        print(idx2word[index_word[1]]," -- ",index_word[0])

print_closest('love',3)

romance  --  0.25692179799079895
lust  --  0.243804469704628
relationship  --  0.21269157528877258


In [5]:
pip install --upgrade tensorflow

  Using cached tensorflow-2.16.2-cp311-cp311-macosx_10_15_x86_64.whl.metadata (4.1 kB)
  Using cached libclang-18.1.1-py2.py3-none-macosx_10_9_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp311-cp311-macosx_10_14_x86_64.whl.metadata (14 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached optree-0.13.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (47 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.16.2-cp311-cp311-macosx_10_15_x86_64.whl (259.6 MB)
Using cached keras-3.7.0-py3-none-any.whl (1.2 MB)
Using cached libclang-18.1.1-py2.